## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import keras
from keras.applications import VGG16 
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [2]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練的 epochs 數量

In [3]:
# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_shape = x_train.shape[1:]
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
# rescale 將 image 做標準化
data_generator = ImageDataGenerator(
    rotation_range=20, # 旋轉
    width_shift_range=0.2, # 平移
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale= 1./255)

In [5]:
test_generator = ImageDataGenerator(rescale = 1./255)

In [6]:
data_generator.fit(x_train)

In [7]:
# 建立 ResNet50 模型
resnet_model = ResNet50(input_shape=input_shape,
                        weights='imagenet',
                        include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


C:\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
activation_50 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_51 (Activation)   (None, 10)                0         
Total params: 24,114,826
Trainable params: 24,061,706
Non-trainable params: 53,120
___________________________________________________________

In [ ]:
model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                        epochs=epochs,
                        validation_data=test_generator.flow(x_test, y_test, batch_size=batch_size),
                        steps_per_epoch=int(len(x_train)//batch_size),
                        validation_steps = 50,
                        workers=4)


Instructions for updating:
Use tf.cast instead.
Epoch 1/30
781/781 [==============================] - 2100s 3s/step - loss: 1.7579 - acc: 0.3935 - val_loss: 2.4299 - val_acc: 0.3734
Epoch 2/30
781/781 [==============================] - 1985s 3s/step - loss: 1.3605 - acc: 0.5147 - val_loss: 1.6795 - val_acc: 0.4600
Epoch 3/30
781/781 [==============================] - 2018s 3s/step - loss: 1.3023 - acc: 0.5429 - val_loss: 2.0435 - val_acc: 0.4284
Epoch 4/30
781/781 [==============================] - 2022s 3s/step - loss: 1.0718 - acc: 0.6260 - val_loss: 1.5097 - val_acc: 0.5301
Epoch 5/30
781/781 [==============================] - 2009s 3s/step - loss: 0.9883 - acc: 0.6605 - val_loss: 1.1101 - val_acc: 0.6409
Epoch 6/30
781/781 [==============================] - 1837s 2s/step - loss: 1.0431 - acc: 0.6393 - val_loss: 1.1526 - val_acc: 0.6016
Epoch 7/30
781/781 [==============================] - 1837s 2s/step - loss: 0.9443 - acc: 0.6720 - val_loss: 1.2058 - val_acc: 0.6158
Epoch 8/30
781

In [ ]:
score = model.evaluate(x_test/255, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])